In [ ]:
#!pip install discord.py
#!pip install nest_asyncio
#!pip install import-ipynb
import nest_asyncio  #to allow nested event loop - else: RuntimeError: Cannot Close a running event loop
nest_asyncio.apply()
import discord
from discord.ext import commands
import logging #to allow logging
#import time
import random
import import_ipynb
import context_finder as cf
from normalize_string import normalize_string
import json
import re

'''logging #so far unused
path = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\logs'
logger_debug = logging.getLogger('discord')
logger_debug.setLevel(logging.DEBUG)
handler_debug = logging.FileHandler(filename=path + r'\discord_debug.log', encoding='utf-8', mode='w')
handler_debug.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(name)s: %(message)s'))
logger_debug.addHandler(handler_debug)

logger_error = logging.getLogger('discord')
logger_error.setLevel(logging.ERROR)
handler_error = logging.FileHandler(filename=path + r'\discord_error.log', encoding='utf-8', mode='w')
handler_error.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(name)s: %(message)s'))
logger_error.addHandler(handler_error)
'''


description = '''A Discord Card Bot for Legends of Runeterra
{{card/keyword}}
'''
bot = commands.Bot(command_prefix='?', description=description)

@bot.event
async def on_ready():
    print('Logged in as')
    print(bot.user.name)
    print(bot.user.id)
    print('------')

@bot.event
async def on_disconnect():
    print('Service got disconnected')
    print('bye bye')
    print('------')
    

#test
#@bot.event
#async def on_member_join(member):
#    print(f'{member} has joined a server')

#test
#@bot.event
#async def on_member_remove(member):
#    print(f'{member} has left a server')


@bot.event
async def on_message(message):
    if message.author.bot:
        return #bot doesn't answer to itself
    #if message.content.startswith('Hello'): await message.channel.send('Hello you wonderful Person!')
        
    search = cf.strings_between_delimiters(message.content)          
    if search: #temp
        found = cf.context_to_messages(search)
        if found:
            for block in found:
                await message.channel.send(block)
        else:
            not_found = cf.to_messages(f"Could not found any cards/core in: [{', '.join(search)}]")
            for block in not_found:
                await message.channel.send(block)
    await bot.process_commands(message)

@bot.command()
async def info(ctx):
    await ctx.send('''Discard Bot
Discord Card Bot for Legends of Runeterra
made by riggedCoinflip''')

#@bot.command()
#async def donate(ctx):
#    await ctx.send('''Hi, thanks for using my Bot!
#if you want to donate me money, please use the following link:
#...''')
#    print ('a')

    
@bot.command()
async def ping(ctx):
    await ctx.send(f'My Ping is: {round(bot.latency * 1000)}ms')
    
@bot.command(aliases = ['op' , 'isop', 'nerf', 'nerfpls', 'plsnerf'])
async def is_op(ctx, *, text):
    responses = [f'Yes, {text} is extremely OP and should be nerfed asap!',
                 f'Are you drunk? {text} is garbage!'] #add more answers
    await ctx.send(f'{random.choice(responses)}')

@bot.command()
async def core_showall(ctx):
    messages =  cf.to_messages([cf.context_to_string(core) for core in cf.all_core()])
    for message in messages:
        await ctx.send(message)
    
@bot.command()
async def cards_showall(ctx):
    messages =  cf.to_messages(cf.all_cards_names())
    for message in messages:
        await ctx.send(message)
        
@bot.command(aliases = ['core', 'content'])
async def card(ctx, *search):
    found = cf.context_to_messages(search)
    if found:
        for block in found:
            await ctx.send(block)
    else:
        not_found = cf.to_messages(f"Could not found any cards/core in: [{', '.join(search)}]")
        for block in not_found:
            await ctx.send(block)
            
@bot.command(aliases = ['random'])
async def rdm(ctx, *, args = []):
    mode, count = "card", 2
    #print(args)
    args = ''.join(args) #join and split are needed to get to a usable function
    #print(args)
    args = args.split(" ")
    #print(args)
    if args:
        for i, arg in enumerate(args[:3]):
            if i == 0:
                mode = normalize_string(arg)
            elif i == 1:
                try:
                    if int(arg) <= 0:
                        print("Use positive Numbers")
                        count = 1
                    elif int(arg) > 40:
                        print("Can not show more than 40 cards this way")
                        count = 40
                    else:
                        count = int(arg)
                except ValueError:
                    print(f"{arg} was not an Integer")
                    count = 1
    if mode =="card":
        #currently allows uncollectibles
        out = []
        #champions = 0 #count of champions
        for i in range(count):
            out.append(random.choice(list(cf.cards['cards'])))
        out = cf.to_messages(out)
        for block in out:
            await ctx.send(block)
    
    if mode == "deck":
        monoregion = 0.1 #Chance of getting a monoregion deck in random deck
        chaos = True #when true, gives much more diverse Cards - when False, high chance of a card appearing more than once in the deck list and more champions
        regions_all = ["Freljord", "Noxus", "Shadow Isles", "Piltover & Zaun", "Demacia", "Ionia"]
        random.shuffle(regions_all)
        regions = [regions_all.pop()]
        if random.random() > monoregion:
            regions.append(regions_all.pop())
        
        
        champions = 0 #not exceed 6
        card_count = 0 #exactly 40
        decklist = [] #[[key1, key2, key3], [amount1, amount2, amount3]]   
        
        if chaos:
            while card_count < 40:
                key = random.choice(list(cf.cards['cards']))
                card = cf.cards['cards'][key]
                is_champ = (card['supertype'] == "Champion")
                print(decklist)
                if card['collectible'] and card['region'] in regions:
                    if decklist:
                        if card in decklist[0]:
                            if decklist[1][decklist[0].index(card)] <= 3:
                                decklist[1][decklist[0].index(card)] += 1
                                card_count += 1
                                if is_champ: champions += 1
                            else: #card already at max
                                continue
                        else:
                            decklist[0].append(card)
                            decklist[1].append(1)
                            card_count += 1
                            if is_champ: champions += 1
                    else:
                        decklist.append([card])
                        decklist.append([1])
                        card_count += 1

            out = [f"{''.join([cf.emote(region, 'region') for region in regions])}"]
            out.append(f"{champions}{cf.emote('Champion', 'type')}")
            for card in zip(decklist[0],decklist[1]):
                out.append(f"{card[1]}: {card[0]['cardname']}")
                   
            out = cf.to_messages(out)
            for block in out:
                await ctx.send(block)
                
                
            
            



'''Exports all Emojis of the Server into a JSON file with: "emoji.name" = str(emoji)'''
@bot.command()
async def emojis_json(ctx):
    emoji_dict = {emoji.name: str(emoji) for emoji in ctx.guild.emojis}  # fill dictionary with values
        
    path = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\source\custom\emojis_import.json'
    with open(path, 'w') as f:
        json.dump(emoji_dict, f, sort_keys=True, indent=4)
        print ("Emojis exported to", path)
        await ctx.send('Fetching Emojis.....Done!')

bot.run('Njc3NTgxODEwNjk1NDA1NTY5.XlxSKg.zgCG6uZRa2XDolGc1DUx9ftDtWs')

importing Jupyter notebook from context_finder.ipynb
importing Jupyter notebook from normalize_string.ipynb
Logged in as
Discard Bot
677581810695405569
------
champion
lastbreath
freljord
champion
champion
elusive
piltoverzaun
champion
Service got disconnected
bye bye
------
Service got disconnected
bye bye
------
Logged in as
Discard Bot
677581810695405569
------
Service got disconnected
bye bye
------
Logged in as
Discard Bot
677581810695405569
------
